# Spectroscopy Function Tests

This notebook provides extensive testing for all functions within the spectroscopy module, ensuring they handle various scenarios correctly and identifying areas for improvement.

### **Core Functions Tested:**
Inhomogenity:
- `normalized_gauss(detunings, fwhm_val)` - Normalized Gaussian distribution for inhomogeneous broadening
- `sample_from_sigma(n_samples, fwhm_val, max_detuning)` - Rejection sampling from Gaussian distribution

Spolvers:
- `check_the_solver(rho)` - Density matrix validation (Hermitian, positive semidefinite, unit trace)

1D Data:
- `compute_1d_polarization` - 1D fixed tau_coh, T_wait:

2D Spectroscopy:
- `get_tau_cohs_and_t_dets_for_T_wait(T_wait, dt, tau_coh_max, t_det_max)` - Time array generation
- `compute_2d_polarization(tau_cohs, t_dets, T_wait, pulse_sequence, params)`

Post Processing:
- `compute_2d_fft_wavenumber(t_dets, tau_cohs, data)` - 2D spectra computation
- `extend_and_plot(...)`
---
**Last Updated:** May 30, 2025

In [ ]:
# =============================
# IMPORT NECESSARY MODULES
# =============================

### Standard scientific libraries
import numpy as np
import time
import warnings
import traceback
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from scipy.stats import norm
from scipy import integrate
import matplotlib.pyplot as plt

### QuTiP quantum toolkit
import qutip as qt
from qutip import mesolve, brmesolve

### Local spectroscopy modules - Core
from qspectro2d.core.system_parameters import SystemParameters
from qspectro2d.core.pulse_sequences import PulseSequence
from qspectro2d.core.solver_fcts import matrix_ODE_paper
from qspectro2d.core.functions_with_rwa import H_int

### Local spectroscopy modules - Calculations
from qspectro2d.spectroscopy.calculations import (
    compute_pulse_evolution,
    check_the_solver,
    compute_1d_polarization,
    compute_2d_polarization,
    parallel_compute_1d_E_with_inhomogenity,
    parallel_compute_2d_E_with_inhomogenity,
    get_tau_cohs_and_t_dets_for_T_wait,
    extract_ift_signal_component,
    _process_single_1d_combination,
    _process_single_2d_combination,
)

### Local spectroscopy modules - Inhomogeneity
from qspectro2d.spectroscopy.inhomogenity import (
    sample_from_sigma,
    normalized_gauss
)

### Local spectroscopy modules - Post-processing
from qspectro2d.spectroscopy.post_processing import (
    compute_1d_fft_wavenumber, 
    compute_2d_fft_wavenumber, 
    extend_and_plot_results
)

### Local visualization modules
from qspectro2d.visualization.plotting import (
    Plot_example_Polarization, 
    plot_2d_el_field, 
    plot_fixed_tau_t, 
    plot_example_evo, 
    plot_1d_frequency_spectrum
)
from qspectro2d.visualization import mpl_tex_settings

print("✓ All imports successful!")

N_atoms_test = 2

## Test 1: inhomogeneous broadening

In [ ]:
### Test sigma function (normalized Gaussian)
print("Testing sigma function (normalized Gaussian distribution)...")

# Test basic functionality
detunings = np.linspace(-5, 5, 40000)
fwhm_val = 2.0
sigma_val = fwhm_val / (2 * np.sqrt(2 * np.log(2)))  # standard deviation from fwhm

### Test sample_from_sigma function
print("\nTesting sample_from_sigma function (rejection sampling)...")
n_samples = 5000

# Generate samples
samples = sample_from_sigma(n_samples, fwhm_val, mu=0, max_detuning=3.0)

# Test statistical properties
sample_mean = np.mean(samples)
sample_std = np.std(samples)
print(f"Sample statistics: mean = {sample_mean:.6f}, std = {sample_std:.6f}")
print(f"Expected: mean ≈ 0, std ≈ {fwhm_val / (2 * np.sqrt(2 * np.log(2))) :.6f}")

### Visualization of sampling results
plt.figure(figsize=(12, 8))

# Plot 2: Sample histogram vs theoretical
plt.subplot(2, 1, 1)
hist_counts, hist_bins, _ = plt.hist(samples, bins=50, density=True, alpha=0.6, 
                                   color='C1', label='Samples')
bin_centers = (hist_bins[:-1] + hist_bins[1:]) / 2
theoretical = normalized_gauss(bin_centers, fwhm_val, mu=0)
# Test normalization (integral should be approximately 1)

plt.plot(bin_centers, theoretical, 'C0', linewidth=2, label='Theoretical')
plt.xlabel(r'Detuning')
plt.ylabel('Probability Density')
plt.title(f'Sample Distribution vs Theoretical\n($N = {n_samples}$ samples)')
plt.legend()

# Plot 3: Cumulative distribution comparison
plt.subplot(2, 1, 2)
sorted_samples = np.sort(samples)
cumulative = np.arange(1, len(sorted_samples) + 1) / len(sorted_samples)
plt.plot(sorted_samples, cumulative, 'C1', label='Sample CDF', linewidth=2)

# Theoretical CDF (using scipy for accuracy)
theoretical_cdf = norm.cdf(sorted_samples, 0, sigma_val)
plt.plot(sorted_samples, theoretical_cdf, 'C0--', label='Theoretical CDF', linewidth=2)
plt.xlabel(r'Detuning')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative Distribution Comparison')
plt.legend()

## Test 2: checking all solverS (compute_pulse_evolution)

In [ ]:
# =============================
# TEST 2: SOLVER VALIDATION (CHECK_THE_SOLVER)
# =============================

### Test check_the_solver function
print("Testing check_the_solver function...")
solvers = ["BR", "ME", "Paper_BR", "Paper_eqs"]
times = np.linspace(-10, 1000, 201)  # time array: 0 to 100 with dt=0.5
for solver in solvers:
    try:
        print(f"Solver: {solver}")
        system_test = SystemParameters(
            ODE_Solver=solver
        )
        res, time_cut = check_the_solver(times, system_test)
        print(f"Time cut: {time_cut}")
    except Exception as e:
        print(f"Error occurred for solver {solver}: {e}")

In [ ]:
from qspectro2d.core.functions_with_rwa import apply_RWA_phase_factors
# =============================
# MULTI-SOLVER DECAY COMPARISON
# =============================

solvers = [
    "BR", 
    "Paper_eqs", 
    #"Paper_BR", # -> THERE IS A BUG here :(
    #"ME"
]
colors  = ['C0', 'C1', 'C2', 'C3']
styles  = ['solid', 'dashed', 'dashdot', 'dotted']

if N_atoms_test == 1:
    psi_0_test = qt.basis(2, 0)  # Initial state |1⟩
elif N_atoms_test == 2:
    psi_0_test = qt.tensor(qt.basis(2, 0), qt.basis(2, 0))

system_current = SystemParameters(
    N_atoms=N_atoms_test,
    t_max=10.0,
    dt=0.1,
    envelope_type="gaussian",
    pulse_fwhm=5.0,
    E0=0.05,
    RWA_laser=True,
    psi_ini=psi_0_test,
    J_cm=0,
    Temp=1e-1,
)
e_ops = system_current.observable_ops
labels = system_current.observable_strs

if N_atoms_test == 2:
    # generate coherence operator for two atoms
    eigenstates = system_current.eigenstates[1]
    e_ops.append(eigenstates[0] * eigenstates[1].dag())
    e_ops.append(eigenstates[0] * eigenstates[2].dag())
    e_ops.append(eigenstates[0] * eigenstates[3].dag())
    e_ops.append(eigenstates[1] * eigenstates[2].dag())
    e_ops.append(eigenstates[1] * eigenstates[3].dag())
    e_ops.append(eigenstates[2] * eigenstates[3].dag())
    labels.append(r'01')
    labels.append(r'02')
    labels.append(r'03')
    labels.append(r'12')
    labels.append(r'13')
    labels.append(r'23')

test_sequence = PulseSequence().from_pulse_specs(system_current, [(0,0,1), ])

times = np.arange(-1 * system_current.fwhms[0], system_current.t_max, system_current.dt)

plt.figure(figsize=(14, 3.5 * len(e_ops)))

for j, e_op in enumerate(e_ops):
    plt.subplot(len(e_ops), 1, j + 1)
    for i, solver in enumerate(solvers):
        system_current.ODE_Solver = solver
        result = compute_pulse_evolution(
            psi_ini=system_current.psi_ini, 
            times=times, 
            pulse_seq=test_sequence, 
            system=system_current
        )
        gamma_0 = system_current.gamma_0
        Gamma = system_current.Gamma
        if solver == "BR":
            gamma_0 = system_current.coupling(system_current.gamma_0)
            Gamma = system_current.coupling(system_current.Gamma)

        states = result.states
        states = [apply_RWA_phase_factors(rho, t, system=system_current) for rho, t in zip(states, times)]
        # print("the traces of the states are for solver ", {solver}, "are: ", [states[i].tr() for i in range(len(states))])
        
        plt.plot(
            times, qt.expect(e_op, states),
            label=rf'{solver}',#  $\gamma_0 = {gamma_0:.3f}$, $\Gamma={Gamma:.3f}$',
            color=colors[i % len(colors)],
            linestyle=styles[i % len(styles)],
            linewidth=2
        )
    plt.xlabel(r'Time $t$ [fs]')
    plt.ylabel(rf'$\langle {labels[j]} \rangle$')
    plt.title(rf'$\langle {labels[j]} \rangle$ for different solvers, $E_0 = {system_current.E0:.3f}$, $T = {system_current.Temp}$')
    plt.legend(fontsize='small', loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlim(times[0], times[-1])

plt.tight_layout()
plt.show()

In [ ]:
a_op, J = system_current.br_decay_channels[1]
print(f"J(omega_A) = {J(system_current.omega_A)}")
a_op, J = system_current.br_decay_channels[0]
print(f"J(omega_A) = {J(0)}")

In [ ]:
from qutip import *

g = basis(2,0)
e = basis(2,1)
g_qtp = e
e_qtp = g
sz = ket2dm(e) - ket2dm(g)
sz_qtp = ket2dm(e_qtp) - ket2dm(g_qtp)
print("comparison of the different basis defintions: sz, sz like i excpect it to be in qutip, how it actually is in qutip", sz, sz_qtp, sigmaz())


In [ ]:
from qspectro2d.core.pulse_functions import E_pulse
print(f"max Et = {np.max(np.abs([E_pulse(t, test_sequence) for t in times]))}")

system_current.H0_diagonalized, system_current.mu_A

In [ ]:
from qutip import *

size = 2 # 2 states |g>, |e>

idx_00 = stacked_index(size,row=0,col=0)
idx_01 = stacked_index(size,row=0,col=1)
idx_10 = stacked_index(size,row=1,col=0)
idx_11 = stacked_index(size,row=1,col=1)

print("Index for |0,0> state:", idx_00)
print("Index for |0,1> state:", idx_01)
print("Index for |1,0> state:", idx_10)
print("Index for |1,1> state:", idx_11)

In [ ]:
rho = qt.Qobj(np.array([
    [0.5, 0.3 + 0.2j],
    [0.3 - 0.2j, 0.5]
]))
print(rho.full())
print(operator_to_vector(rho))  # zeigt dir: wer ist wo

In [ ]:
from qutip import QobjEvo, liouvillian
from qspectro2d.core.functions_with_rwa import H_int
from qspectro2d.core.solver_fcts import R_paper

H_free = system_current.H0_diagonalized  # already includes the RWA, if present!
print(H_free, flush=True)

# Fix the QobjEvo definition
H_int_evo = QobjEvo(
    lambda t, args=None: H_free + H_int(t, test_sequence, system_current)
)  # also add H_int, with potential RWA

L = liouvillian(H_int_evo(1)) + R_paper(system=system_current)
L
print("the combined liouvillian from L = liouvillian(H_int_evo(1)) + R_paper(system=system_current):", L, flush=True)
from qspectro2d.core.solver_fcts import matrix_ODE_paper
print("the full liouvillian from paper", matrix_ODE_paper(1, test_sequence, system_current))

In [ ]:
from qutip import QobjEvo
from qspectro2d.core.functions_with_rwa import H_int
H_free = system_current.H0_diagonalized  # already includes the RWA, if present!
print(H_free, flush=True)

# Fix the QobjEvo definition
H_int_evo = QobjEvo(
    lambda t, args=None: H_free + H_int(t, test_sequence, system_current)
)  # also add H_int, with potential RWA

# =============================
# VISUALIZE QOBJEVO STRENGTH OVER TIME
# =============================

### Create time array for visualization
t_vis = np.linspace(-2 * system_current.fwhms[0], system_current.t_max, 500)

### Calculate Hamiltonian matrix elements over time
H_strengths = []
H_interaction_strengths = []
for t in t_vis:
    # Total Hamiltonian at time t
    H_total = H_int_evo(t)
    
    # Interaction part only
    H_interaction = H_int(t, test_sequence, system_current)
    
    # Calculate various strength measures
    H_strengths.append(np.linalg.norm(H_total.full()))  # Frobenius norm
    H_interaction_strengths.append(np.linalg.norm(H_interaction.full()))

H_strengths = np.array(H_strengths)
H_interaction_strengths = np.array(H_interaction_strengths)

### Create visualization plots
plt.figure(figsize=(12, 8))

### Plot 1: Total Hamiltonian strength
plt.subplot(2, 2, 1)
plt.plot(t_vis, H_strengths, label=r'$||H_{\mathrm{total}}(t)||_F$', color='C0', linestyle='solid')
plt.xlabel(r'Time $t$ [fs]')
plt.ylabel(r'Hamiltonian Strength')
plt.title(r'Total Hamiltonian Frobenius Norm vs Time')
plt.legend()

### Plot 2: Interaction Hamiltonian strength
plt.subplot(2, 2, 2)
plt.plot(t_vis, H_interaction_strengths, label=r'$||H_{\mathrm{int}}(t)||_F$', color='C1', linestyle='solid')
plt.xlabel(r'Time $t$ [fs]')
plt.ylabel(r'Interaction Strength')
plt.title(r'Interaction Hamiltonian Frobenius Norm vs Time')
plt.legend()

### Plot 3: Both on same plot (normalized)
plt.subplot(2, 2, 3)
H_max = np.max(H_strengths)
H_int_max = np.max(H_interaction_strengths)
plt.plot(t_vis, H_strengths / H_max, label=r'$||H_{\mathrm{total}}(t)||_F$ (norm.)', color='C0', linestyle='solid')
plt.plot(t_vis, H_interaction_strengths / H_int_max, label=r'$||H_{\mathrm{int}}(t)||_F$ (norm.)', color='C1', linestyle='dashed')
plt.xlabel(r'Time $t$ [fs]')
plt.ylabel(r'Normalized Strength')
plt.title(r'Normalized Hamiltonian Strengths')
plt.legend()

### Plot 4: Diagonal vs off-diagonal elements strength
diagonal_strengths = []
off_diagonal_strengths = []
for t in t_vis:
    H_total = H_int_evo(t).full()
    diagonal_strengths.append(np.linalg.norm(np.diag(H_total)))
    off_diagonal_strengths.append(np.linalg.norm(H_total - np.diag(np.diag(H_total))))

plt.subplot(2, 2, 4)
plt.plot(t_vis, diagonal_strengths, label=r'Diagonal elements', color='C2', linestyle='solid')
plt.plot(t_vis, off_diagonal_strengths, label=r'Off-diagonal elements', color='C3', linestyle='dashed')
plt.xlabel(r'Time $t$ [fs]')
plt.ylabel(r'Element Strength')
plt.title(r'Diagonal vs Off-diagonal Hamiltonian Elements')
plt.legend()

plt.tight_layout()
plt.show()

### Print some statistics
print(f"Maximum total Hamiltonian strength: {np.max(H_strengths):.3f}")
print(f"Maximum interaction strength: {np.max(H_interaction_strengths):.3f}")
print(f"Ratio of interaction to total (max): {np.max(H_interaction_strengths)/np.max(H_strengths):.3f}")
print(f"Time of maximum interaction: {t_vis[np.argmax(H_interaction_strengths)]:.3f} fs")



In [ ]:
# =============================
# VISUALIZE POWER SPECTRUM FUNCTION
# =============================
print(system_current.br_decay_channels)
### Extract the power spectrum function and its parameters from the system_current
current_P_fct = system_current.br_decay_channels[1][1]  # This is the power spectrum function for the first decay channel

### Create frequency range for visualization
w_max = 1 * system_current.omega_A  # Maximum frequency
w_range = np.linspace(-1*w_max, w_max, 1000)

### Calculate power spectrum values
power_values = [current_P_fct(w) for w in w_range]
power_values = np.array(power_values)

### Create the plot
plt.figure(figsize=(10, 6))

### Plot the power spectrum
plt.plot(w_range / system_current.omega_A, power_values, 
         label=r'$S(\omega)$', linestyle='solid', color='C0')

if np.any(np.imag(power_values) != 0):
    plt.plot(w_range / system_current.omega_A, np.imag(power_values), 
             label=r'$S(\omega)$ (Imag)', linestyle='dashed', color='C1')

### Add vertical lines at characteristic frequencies
plt.axvline(x=-1, color='red', linestyle='--', alpha=0.7, label=r'$\omega_{at}$')

### Formatting
plt.xlabel(r'Frequency $\omega / \omega_{at}$')
plt.ylabel(r'power Spectrum $S(\omega)$')
plt.title(f'power Spectrum for' + 
          f'Bath: paper, T = {system_current.Temp:.1e}, ' + 
          r'$\gamma_{\mathrm{relax}} = $' + f'{system_current.gamma_0:.1e}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Print some key values
print(f"power spectrum at ω = 0: {current_P_fct(0):.3e}")
print(f"power spectrum at ω = ω_at: {current_P_fct(system_current.omega_A):.3e}")
print(f"power spectrum at ω = cutoff: {current_P_fct(system_current.cutoff):.3e}")
print(f"\nChannel arguments used:")

## Test 3: compute_1d_polarization

In [ ]:
# Test compute_1d_polarization for dummy variables

# Define dummy input values
tau_coh_test = 300.0
T_wait_test  = 1000.0
phi_0_test   = 0.0
phi_1_test   = 0.0*np.pi / 2
system_test  = SystemParameters(
    N_atoms=N_atoms_test,
    t_max=tau_coh_test + T_wait_test + 600.0,  # Maximum time for simulation
    dt=2,      # Time step
    envelope_type="gaussian",  # Use Gaussian pulse envelope
    pulse_fwhm=15.0,
    omega_A_cm=16000.0,
    E0=0.005,
    ODE_Solver="BR",  # Use Paper BR solver
    RWA_laser= True,  # Use RWA for laser
#    gamma_phi=0.20,  # No dephasing
#    gamma_0=0.1
)  # Use default parameters
t_max = system_test.t_max  # Maximum time for simulation
# Create time array from -fwhm to t_max with dt
times_test   = np.arange(-20* system_test.fwhms[0], t_max, system_test.dt)  # Time array from 0 to t_max with dt
print(f"Testing compute_1d_polarization with parameters:\n")
print(f"The times run from {times_test[0]:.2f} to {times_test[-1]:.2f} with dt = {system_test.dt:.3f}")
time_cut = np.inf
kwargs = {
    "time_cut": time_cut,  # to avoid numerical issues
#    "tau_coh": tau_coh_test,
#    "T_wait": T_wait_test,
    "function": "P" # Polarization function 
}

try:
    result = compute_1d_polarization(
        tau_coh=tau_coh_test,
        T_wait=T_wait_test,
        phi_0=phi_0_test,
        phi_1=phi_1_test,
        times=times_test,
        system=system_test,
        **kwargs
    )
    # now also make a resutl = parallel_compute_1d_E_with_inhomogenity(...) 
    
    ### Test parallel_compute_1d_E_with_inhomogenity with inhomogeneous broadening
    kwargs_parallel = {
        "n_freqs":10,  # Number of phases for phase cycling (should be 4 for IFT)
        # to avoid numerical issues
        "max_workers": 8,  # Number of parallel workers (adjust based on your CPU)
    }
    kwargs_parallel.update({
        "function": "E_{k_s}"
  
        }
        )  # Polarization function    
    result_parallel = parallel_compute_1d_E_with_inhomogenity(
        n_phases=4,  # Number of frequencies for inhomogeneous broadening
        tau_coh=tau_coh_test,
        T_wait=T_wait_test,
        times=times_test,
        system=system_test,
        **kwargs_parallel
    )
    print("✓ parallel_compute_1d_E_with_inhomogenity executed successfully.")
    
    # Plot the data for a fixed tau_coh and T_wait
    plot_fixed_tau_t(
        result[0], result[1], **kwargs
    )
    plot_fixed_tau_t(
        result_parallel[0], result_parallel[1], **kwargs_parallel
    )
    print("✓ compute_1d_polarization executed successfully.")
except Exception as e:
    print("❌ compute_1d_polarization failed:", e)

In [ ]:
# =============================
# TEST: extract_ift_signal_component Function
# =============================
print("Testing extract_ift_signal_component function...")
# TODO write an appropriate test for this function

In [ ]:
# =============================
# TEST: extract_ift_signal_component Function
# =============================
print("Testing extract_ift_signal_component function...")

### Setup phase cycling parameters
n_phases      = 4  # typical for phase cycling
phases        = np.array([0, np.pi/2, np.pi, 3*np.pi/2])  # example phase values
signal        = np.zeros((n_phases, n_phases), dtype=object)  # store the signal data

### Define kwargs for processing (matching previous test parameters)
kwargs = {
    "time_cut": time_cut,  # to avoid numerical issues
    "plot_example_Polarization": False,  # Disable plots for batch processing
    "function": "E_{k_s}"  # Polarization function 
}

# =============================
# GLOBAL WORKER FUNCTION FOR MULTIPROCESSING
# =============================

def process_phase_combination_worker(args):
    """
    Global worker function for ProcessPoolExecutor
    Must be defined at module level for pickling
    """
    import numpy as np
    from qspectro2d.spectroscopy.calculations import _process_single_1d_combination
    
    phi1_idx, phi1, phi2_idx, phi2, tau_coh_val, T_wait_val, times_arr, system_params, processing_kwargs = args
    
    try:
        result_temp = _process_single_1d_combination(
            phi1=phi1,
            phi2=phi2,
            tau_coh=tau_coh_val,
            T_wait=T_wait_val,
            times=times_arr,
            system=system_params,
            **processing_kwargs
        )
        
        if result_temp is not None and len(result_temp) >= 2:
            return (phi1_idx, phi2_idx, result_temp[1], result_temp[0])  # Return data and times
        else:
            return (phi1_idx, phi2_idx, None, None)
            
    except Exception as e:
        print(f"❌ Exception for phase ({phi1_idx}, {phi2_idx}): {e}", flush=True)
        return (phi1_idx, phi2_idx, None, None)

# =============================
# PARALLEL PHASE COMBINATION PROCESSING WITH PROCESSPOOL
# =============================

### Prepare arguments for all phase combinations (with serializable parameters)
phase_args = [
    (phi1_idx, phi1, phi2_idx, phi2, tau_coh_test, T_wait_test, times_test, system_test, kwargs)
    for phi1_idx, phi1 in enumerate(phases)
    for phi2_idx, phi2 in enumerate(phases)
]

print(f"Processing {len(phase_args)} phase combinations with ProcessPoolExecutor...")

### Parallel execution with ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=16) as executor:  # Adjust max_workers based on CPU cores
    results = list(executor.map(process_phase_combination_worker, phase_args))
    

### Fill the signal array with results
times_result = None  # Store time array from first successful result
successful_count = 0

for phi1_idx, phi2_idx, data_val, times_val in results:
    if data_val is not None:
        signal[phi1_idx, phi2_idx] = data_val
        successful_count += 1
        if times_result is None:  # Store times from first successful calculation
            times_result = times_val
    else:
        signal[phi1_idx, phi2_idx] = None
        print(f"❌ No result for phase combination ({phi1_idx}, {phi2_idx})")

print(f"✓ Successfully processed {successful_count}/{len(phase_args)} phase combinations")

# =============================
# TEST SIGNAL EXTRACTION
# =============================

try:
    ### Extract the IFT signal component (e.g., rephasing or non-rephasing)
    extracted_signal = extract_ift_signal_component(
        signal, 
        phases=phases, 
        component=(-1, 1, 0)  # Example component specification
    )

    if extracted_signal is not None and times_result is not None:
        ### Plot the extracted signal
        plot_fixed_tau_t(
            times_result, extracted_signal, **kwargs
        )
        print(f"✓ extract_ift_signal_component succeeded!")
        print(f"Signal shape: {np.array(extracted_signal).shape}")
        print(f"First 5 values: {extracted_signal[:5]}")
    else:
        print("❌ extract_ift_signal_component returned None (no valid results)")
        
except Exception as e:
    print(f"❌ extract_ift_signal_component failed: {e}")
    traceback.print_exc()

In [ ]:
t_freqs, data_freqs = compute_1d_fft_wavenumber(
    result[0], result[1])

plot_1d_frequency_spectrum(
    t_freqs, data_freqs,
    component="abs")

In [ ]:
# Test compute_1d_polarization for dummy variables
kwargs = {
    "time_cut": time_cut,  # to avoid numerical issues
#    "tau_coh": tau_coh_test,
#    "T_wait": T_wait_test,
    "plot_example_Polarization": True,  # Enable example polarization plot
    "function": "P" # Polarization function 
}

try:
    times_plot, P_full, P_only0, P_only1, P_only2 = compute_1d_polarization(
        tau_coh=tau_coh_test,
        T_wait=T_wait_test,
        phi_0=phi_0_test,
        phi_1=phi_1_test,
        times=times_test,
        system=system_test,
        **kwargs  # Unpack the kwargs dictionary with **
    )
    # Plot the data for a fixed tau_coh and T_wait
    Plot_example_Polarization(
        times_plot, P_full, P_only0, P_only1, P_only2, **kwargs  # Unpack kwargs here too
    )
    print("✓ compute_1d_polarization executed successfully.")
except Exception as e:
    print("❌ compute_1d_polarization failed:", e)

In [ ]:
### Create test system and time arrays
print("\nSetting up test parameters...")

# Plot an example evolution:
kwargs = {
    "plot_example": True,
    "time_cut": time_cut,
}

try:
    res = compute_1d_polarization(
        tau_coh=tau_coh_test,
        T_wait=T_wait_test,
        phi_0=phi_0_test,
        phi_1=phi_1_test,
        times=times_test,
        system=system_test,
        **kwargs  # Unpack the kwargs dictionary with **
    )
    times_plot, data_expectations, pulse_seq_f, additional_info = res[:]

    plot_example_evo(times_plot, data_expectations, pulse_seq_f, tau_coh=tau_coh_test, T_wait=T_wait_test, system=system_test, additional_info=additional_info)
    print(f"Example evolution plot created successfully.")
except Exception as e:
    print(f"Error during compute_2d_polarization: {e}")


In [ ]:
kwargs = {
    "time_cut": time_cut,  # to avoid numerical issues
#    "tau_coh": tau_coh_test,
#    "T_wait": T_wait_test,
    "function": "E_{ks}",  # Electric field function
    "n_freqs":10,
}

try:
    result = parallel_compute_1d_E_with_inhomogenity(n_freqs=kwargs["n_freqs"], n_phases=4, tau_coh=tau_coh_test, T_wait=T_wait_test, times=times, system=system_test, **kwargs)
    # Plot the data for a fixed tau_coh and T_wait
    plot_fixed_tau_t(
        result[0], result[1], **kwargs
    )
    print("✓ compute_1d_polarization executed successfully.")
except Exception as e:
    print("❌ compute_1d_polarization failed:", e)

t_freqs, data_freqs = compute_1d_fft_wavenumber(
    result[0], result[1])

plot_1d_frequency_spectrum(
    t_freqs, data_freqs,
    type="abs"
)

## Test 4: get_tau_cohs_and_t_dets_for_T_wait

In [ ]:
# =============================
# TEST 1: TIME ARRAY CALCULATIONS
# =============================

### Test get_tau_cohs_and_t_dets_for_T_wait function
print("Testing get_tau_cohs_and_t_dets_for_T_wait function...")

# Test basic functionality with reasonable parameters
times = np.linspace(-2, 100, 201)  # time array: 0 to 100 with dt=0.5
T_wait = 50.0  # waiting time

tau_cohs, t_dets = get_tau_cohs_and_t_dets_for_T_wait(times, T_wait)

print(f"Generated time arrays:")
print(
    f"  Input times: {len(times)} points, range [{times[0]:.2f}, {times[-1]:.2f}], dt = {times[1]-times[0]:.3f}"
)
print(f"  T_wait = {T_wait}")
print(
    f"  tau_cohs: {len(tau_cohs)} points, range [{tau_cohs[0]:.2f}, {tau_cohs[-1]:.2f}]"
)
print(f"  t_dets: {len(t_dets)} points, range [{t_dets[0]:.2f}, {t_dets[-1]:.2f}]")

# Test array properties
assert len(tau_cohs) > 0, "tau_cohs array is empty"
assert len(t_dets) > 0, "t_dets array is empty"
assert len(tau_cohs) == len(t_dets), "tau_cohs and t_dets should have same length"
assert tau_cohs[0] >= 0, "tau_cohs should start from non-negative value"
assert t_dets[0] >= 0, "t_dets should start from non-negative value"
assert (
    t_dets[-1] <= times[-1]
), f"t_dets exceeds maximum time: {t_dets[-1]} > {times[-1]}"


# Test with T_wait = t_max (edge case)
print("\nTesting T_wait = t_max (edge case):")
T_wait_max = times[-1]
tau_edge, t_det_edge = get_tau_cohs_and_t_dets_for_T_wait(times, T_wait_max)
print(
    f"  T_wait = t_max = {T_wait_max}: tau_cohs length = {len(tau_edge)}, t_dets length = {len(t_det_edge)}"
)

if len(tau_edge) > 0:
    print(f"    Values: tau_coh = {tau_edge}, t_det = {t_det_edge}")

# Test with T_wait > t_max (should return empty arrays)
print("\nTesting T_wait > t_max (should return empty):")
T_wait_large = times[-1] + 10.0
tau_empty, t_det_empty = get_tau_cohs_and_t_dets_for_T_wait(times, T_wait_large)
print(
    f"  T_wait = {T_wait_large} > t_max = {times[-1]}: tau_cohs length = {len(tau_empty)}, t_dets length = {len(t_det_empty)}"
)
assert len(tau_empty) == 0, "Should return empty array when T_wait > t_max"
assert len(t_det_empty) == 0, "Should return empty array when T_wait > t_max"

### Test with single time point
print("\nTesting single time point:")
times_single = np.array([0.0])
tau_single, t_det_single = get_tau_cohs_and_t_dets_for_T_wait(times_single, 0.0)
print(f"  Single time point: tau_cohs = {tau_single}, t_dets = {t_det_single}")

### Visualization of time arrays
print("\nCreating visualizations...")
plt.figure(figsize=(14, 10))

### Plot 1: Time arrays for different T_wait values
plt.subplot(2, 3, 1)
T_wait_vis = [-10.0, 10.0, 25.0, 40.0, 60.0, 80.0]
colors = plt.cm.viridis(np.linspace(0, 1, len(T_wait_vis)))

for i, T_w in enumerate(T_wait_vis):
    tau_vis, t_det_vis = get_tau_cohs_and_t_dets_for_T_wait(times, T_w)
    if len(tau_vis) > 0:
        plt.plot(
            tau_vis,
            np.full_like(tau_vis, i),
            "o",
            color=colors[i],
            markersize=2,
            label=rf"$\tau_{{\text{{coh}}}}$ ($T_{{\text{{wait}}}}={T_w}$)",
            alpha=0.7,
        )
        plt.plot(
            t_det_vis,
            np.full_like(t_det_vis, i + 0.1),
            "s",
            color=colors[i],
            markersize=2,
            alpha=0.7,
            label=rf"$t_{{\text{{det}}}}$ ($T_{{\text{{wait}}}}={T_w}$)",
        )
plt.xlabel(r"Time $t$")
plt.ylabel(r"$T_{\text{wait}}$ Value Index")
plt.title(r"Time Arrays for Different $T_{\text{wait}}$ Values")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
plt.grid(True, alpha=0.3)

### Plot 2: Array length vs T_wait
plt.subplot(2, 3, 2)
T_wait_range = np.linspace(5, 95, 50)
array_lengths = []

for T_w in T_wait_range:
    tau_test, t_det_test = get_tau_cohs_and_t_dets_for_T_wait(times, T_w)
    array_lengths.append(len(tau_test))

plt.plot(T_wait_range, array_lengths, "C0o-", markersize=3)
plt.xlabel(r"$T_{\text{wait}}$")
plt.ylabel(r"Array Length")
plt.title(r"Array Length vs $T_{\text{wait}}$")
plt.grid(True, alpha=0.3)

### Plot 3: Time coverage analysis
plt.subplot(2, 3, 3)
tau_coverage = []
t_det_coverage = []

for T_w in T_wait_range:
    tau_test, t_det_test = get_tau_cohs_and_t_dets_for_T_wait(times, T_w)
    tau_max = tau_test[-1] if len(tau_test) > 0 else 0
    t_det_max = t_det_test[-1] if len(t_det_test) > 0 else 0
    tau_coverage.append(tau_max)
    t_det_coverage.append(t_det_max)

plt.plot(T_wait_range, tau_coverage, "C0o-", label=r"Max $\tau_{\text{coh}}$", markersize=3)
plt.plot(T_wait_range, t_det_coverage, "C1s-", label=r"Max $t_{\text{det}}$", markersize=3)
plt.plot(
    T_wait_range, 
    times[-1] - T_wait_range, 
    "k--", 
    alpha=0.5, 
    label=r"Expected max $\tau_{\text{coh}}$"
)
plt.plot(
    T_wait_range, 
    np.full_like(T_wait_range, times[-1]), 
    "r--", 
    alpha=0.5, 
    label=r"$t_{\text{max}}$"
)
plt.xlabel(r"$T_{\text{wait}}$")
plt.ylabel(r"Time")
plt.title(r"Time Coverage Analysis")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ All time array calculation tests passed!")


## Test 5: compute_2d_polarization

In [ ]:
### Test compute_2d_polarization function
print("Testing compute_2d_polarization function...")

### Create test system and time arrays
print("\nSetting up test parameters...")
system_test = SystemParameters(
    t_max=10.0,  # Maximum time for simulation
    dt=2,      # Time step
    envelope_type='gaussian',  # Envelope type for pulses
    ODE_Solver='Paper_eqs',  # ODE solver to useç
    N_atoms=N_atoms_test,
)
t_max = system_test.t_max
times_test = np.arange(-system_test.fwhms[0], t_max, system_test.dt)  # Dense time array for accurate calculation

# Test parameters for 2DES simulation
phi_0 = 0.0             # Phase of first pulse
phi_1 = np.pi/4         # Phase of second pulse
T_wait = 0        # Waiting time between pulses
time_cut = np.inf       # No time cut for this test

print(f"System parameters:")
print(f"  Time array: {len(times_test)} points, dt = {times_test[1]-times_test[0]:.3f}")

### Test T_wait near time limits
print("\nTesting T_wait near time limits:")

T_wait_edge_cases = [0]# [0, t_max/2, t_max]
for T_w in T_wait_edge_cases:
    try:
        t_det, tau_coh, data = compute_2d_polarization(
            T_wait=T_w, phi_0=phi_0, phi_1=phi_1, times=times_test, system=system_test, time_cut=time_cut
        )
        plot_2d_el_field((t_det, tau_coh, data), use_custom_colormap=True)
        print(f"  T_wait={T_w:5.1f} ({T_w/t_max:.1%} of t_max): ✓ Success, shape {data.shape}")
    except Exception as e:
        print(f"  T_wait={T_w:5.1f}: ❌ Failed - {e}")

    
"""### Test with time_cut parameter
print("\nTesting time_cut parameter:")
time_cuts = []# [np.inf, t_max/2, t_max]
for tc in time_cuts:
    try:
        t_det, tau_coh, data = compute_2d_polarization(
            T_wait=T_wait, phi_0=phi_0, phi_1=phi_1, times=times_test, system=system_test, time_cut=tc
        )
        plot_2d_el_field((t_det, tau_coh, data), use_custom_colormap=True)

        effective_max_time = np.max(tau_coh) + T_wait + np.max(t_det)
        print(f"  time_cut={tc:5.1f}: ✓ Success, effective_max_time={effective_max_time:.1f}")
    except Exception as e:
        print(f"  time_cut={tc:5.1f}: ❌ Failed - {e}")"""


In [ ]:
# =============================
# TEST: extract_ift_signal_component Function (2D)
# =============================
print("Testing extract_ift_signal_component function for 2D spectroscopy...")

### Setup phase cycling parameters
n_phases      = 4  # typical for phase cycling
phases        = np.array([0, np.pi/2, np.pi, 3*np.pi/2])  # example phase values
signal        = np.zeros((n_phases, n_phases), dtype=object)  # store the signal data

### Define kwargs for 2D processing
kwargs = {
    "time_cut": time_cut,  # to avoid numerical issues
    "function": "P"  # Polarization function 
}

### Generate test 2D time arrays (tau_cohs and t_dets)
T_wait_2d        = 0.0  # Fixed waiting time for 2D test

tau_cohs, t_dets = get_tau_cohs_and_t_dets_for_T_wait(
    times=times_test,
    T_wait=T_wait_2d, 
)

print(f"Generated 2D time arrays: tau_cohs ({len(tau_cohs)} points), t_dets ({len(t_dets)} points)")

# =============================
# PARALLEL PHASE COMBINATION PROCESSING FOR 2D
# =============================

def process_2d_phase_combination(args):
    """Process a single 2D phase combination for parallel execution"""
    phi1_idx, phi1, phi2_idx, phi2, processing_kwargs = args
    
    try:
        result_temp = _process_single_2d_combination(
            phi1=phi1,
            phi2=phi2,
            T_wait=T_wait_2d,
            times=times_test,
            system=system_test,
            **processing_kwargs
        )
        
        if result_temp is not None and len(result_temp) >= 3:
            # result_temp should contain [tau_cohs, t_dets, data]
            return (phi1_idx, phi2_idx, result_temp[2], result_temp[0], result_temp[1])
        else:
            return (phi1_idx, phi2_idx, None, None, None)
            
    except Exception as e:
        print(f"❌ Exception for 2D phase ({phi1_idx}, {phi2_idx}): {e}", flush=True)
        return (phi1_idx, phi2_idx, None, None, None)

### Prepare arguments for all phase combinations
phase_args = [
    (phi1_idx, phi1, phi2_idx, phi2, kwargs)
    for phi1_idx, phi1 in enumerate(phases[:n_phases])
    for phi2_idx, phi2 in enumerate(phases[:n_phases])
]

print(f"Processing {len(phase_args)} 2D phase combinations in parallel...")

### Parallel execution
with ProcessPoolExecutor(max_workers=16) as executor:
    results = list(executor.map(process_2d_phase_combination, phase_args))

### Store results and capture time arrays
tau_cohs_result = None  # Store tau_cohs from first successful result
t_dets_result   = None  # Store t_dets from first successful result

for phi1_idx, phi2_idx, data_val, tau_cohs_val, t_dets_val in results:
    if data_val is not None:
        signal[phi1_idx, phi2_idx] = data_val
        if tau_cohs_result is None:  # Store time arrays from first successful calculation
            tau_cohs_result = tau_cohs_val
            t_dets_result   = t_dets_val
    else:
        signal[phi1_idx, phi2_idx] = None
        print(f"❌ No result for 2D phase combination ({phi1_idx}, {phi2_idx})", flush=True)

# =============================
# TEST 2D SIGNAL EXTRACTION
# =============================

try:
    ### Extract the IFT signal component (e.g., rephasing or non-rephasing)
    extracted_signal = extract_ift_signal_component(
        signal, 
        phases=phases, 
        component=(-1, 1, 0)  # Example component specification
    )
    
    print(f"Extracted signal type: {type(extracted_signal)}")
    if hasattr(extracted_signal, 'shape'):
        print(f"Extracted signal shape: {extracted_signal.shape}")
    
    if extracted_signal is not None and tau_cohs_result is not None and t_dets_result is not None:
        ### Plot the 2D extracted signal
        plot_2d_el_field(
            (tau_cohs_result, t_dets_result, extracted_signal)
        )
        
        ### Compute 2D FFT
        nu_ts, nu_taus, data_freqs = compute_2d_fft_wavenumber(
            tau_cohs_result, t_dets_result, extracted_signal
        )

        print(f"✓ extract_ift_signal_component for 2D succeeded!")
        print(f"2D signal shape: {np.array(extracted_signal).shape}")
    else:
        print("❌ extract_ift_signal_component returned None (no valid 2D results)")
        
except Exception as e:
    print(f"❌ extract_ift_signal_component for 2D failed: {e}")
    traceback.print_exc()

In [ ]:
### Create test system and time arrays
print("\nSetting up test parameters...")
system_test = SystemParameters(
    N_atoms=N_atoms_test,  # Number of atoms
    t_max=40.0,  # Maximum time for simulation
    dt=1,      # Time step
    envelope_type='gaussian',  # Envelope type for pulses
    pulse_fwhm=10.0,  # Pulse fwhm,
    ODE_Solver='Paper_eqs',  # ODE solver to use
)
t_max = system_test.t_max
timtimes_test = np.arange(-system_test.fwhms[0], t_max, system_test.dt)  # Dense time array for accurate calculation

# Plot an example evolution:
tau_coh_ex = t_max / 3
kwargs = {
    "plot_example": True,
    "plot_example_for_tau": tau_coh_ex,
}

try:
    res = compute_2d_polarization(
        T_wait=T_wait, phi_0=phi_0, phi_1=phi_1, times=times_test, system=system_test, time_cut=time_cut, **kwargs
    )
    times_plot, data_expectations, pulse_seq_f, additional_info = res[:]

    plot_example_evo(times_plot, data_expectations, pulse_seq_f, tau_coh_ex, T_wait=T_wait, system=system_test)
    print(f"Example evolution plot created successfully.")
except Exception as e:
    print(f"Error during compute_2d_polarization: {e}")

In [ ]:
### Create test system and time arrays
print("\nSetting up test parameters...")
system_test = SystemParameters(
    N_atoms=N_atoms_test,  # Number of atoms
    t_max=100.0,  # Maximum time for simulation
    dt=0.1,      # Time step
    envelope_type='gaussian',  # Envelope type for pulses
    pulse_fwhm=10.0,  # Pulse fwhm
)

n_freqs = 1
n_phases = 3
t_max = system_test.t_max
times_test = np.arange(-system_test.fwhms[0], t_max, system_test.dt)  # Dense time array for accurate calculation

plot_args = {
    "domain": "freq",
    "component": "real",
    "save": False,
    #"use_custom_colormap": True,
    # "section": (1, 2, 1, 2),  # Plot the first section
    #"system": system_test,
}

try:
    data_list = parallel_compute_2d_E_with_inhomogenity(
        n_freqs=n_freqs, n_phases=n_phases, times_T=[T_wait], times=times_test, system=system_test
    )
    
    extend_and_plot_results(data_list, times_T=[T_wait], times=times_test, extend_for=(1,1), **plot_args)

except Exception as e:
    print(f"Error during compute_2d_polarization: {e}")


In [ ]:
plot_args = {
    "domain": "freq",
    "component": "imag",
    "save": False,
    #"use_custom_colormap": True,
    "section": (1.5, 1.8, 1.5, 1.8),  # Plot the first section
    #"system": system_test,
}
extend_and_plot_results(data_list, times_T=[T_wait], times=times_test, extend_for=(1,2), **plot_args)


In [ ]:
# =============================
# TEST 4: PHYSICS VALIDATION
# =============================

print("\n--- TEST 4: Physics validation ---")
### Test phase dependence (should show different signals for different phases)
print("\nTesting phase dependence:")
try:
    # Compare in-phase vs out-of-phase signals
    _, _, data_in_phase = compute_2d_polarization(
        T_wait=5.0, phi_0=0, phi_1=0, times=times_test[:101], system=system_test, time_cut=20.0
    )
    _, _, data_out_phase = compute_2d_polarization(
        T_wait=5.0, phi_0=0, phi_1=np.pi, times=times_test[:101], system=system_test, time_cut=20.0
    )
    
    rms_in = np.sqrt(np.mean(data_in_phase**2))
    rms_out = np.sqrt(np.mean(data_out_phase**2))
    phase_difference = np.sqrt(np.mean((data_in_phase - data_out_phase)**2))
    
    print(f"  In-phase RMS: {rms_in:.2e}")
    print(f"  Out-phase RMS: {rms_out:.2e}")
    print(f"  Phase difference RMS: {phase_difference:.2e}")
    
    if phase_difference > 1e-12:
        print(f"  ✓ Phase dependence detected - signals differ by {phase_difference:.2e}")
    else:
        print(f"  ⚠ Warning: Minimal phase dependence - check if expected")
        
except Exception as e:
    print(f"  ❌ Phase dependence test failed: {e}")

### Test T_wait dependence
print("\nTesting T_wait dependence:")
try:
    T_waits = [2.0, 8.0]
    data_short = compute_2d_polarization(
        T_wait=T_waits[0], phi_0=0, phi_1=0, times=times_test[:101], system=system_test, time_cut=15.0
    )[2]
    data_long = compute_2d_polarization(
        T_wait=T_waits[1], phi_0=0, phi_1=0, times=times_test[:101], system=system_test, time_cut=25.0
    )[2]

    # Compare signal magnitudes (should decay with longer T_wait due to relaxation)
    rms_short = np.sqrt(np.mean(data_short**2))
    rms_long = np.sqrt(np.mean(data_long**2))
    
    print(f"  T_wait={T_waits[0]}: RMS={rms_short:.2e}")
    print(f"  T_wait={T_waits[1]}: RMS={rms_long:.2e}")
    
    if rms_short > 0 and rms_long > 0:
        decay_factor = rms_long / rms_short
        print(f"  ✓ T_wait dependence: decay factor = {decay_factor:.3f}")
        if decay_factor < 1.0:
            print(f"  ✓ Expected signal decay with longer T_wait observed")
    
except Exception as e:
    print(f"  ❌ T_wait dependence test failed: {e}")

## Test 6: compute_2d_fft_wavenumber Function

This section tests the `compute_2d_fft_wavenumber` function with a 2D Gaussian to understand the shape mismatch issue that causes the IndexError.

In [ ]:
# =============================
# TEST 2D FFT FUNCTION WITH GAUSSIAN
# =============================

### Create test data: 2D Gaussian
N_t, N_tau = 5, 5
ts = np.linspace(0, 100, N_t)        # 0-100 fs
taus = np.linspace(0, 80, N_tau)     # 0-80 fs

# Create 2D Gaussian centered at (40 fs, 60 fs)
tau_center, t_center = 40, 60
sigma_tau, sigma_t = 10, 15

# Create meshgrid for 2D Gaussian
TAU, T = np.meshgrid(taus, ts, indexing='ij')  # Shape: (N_tau, N_t)
gaussian_2d = np.exp(-((TAU - tau_center)**2 / (2 * sigma_tau**2) + 
                       (T - t_center)**2 / (2 * sigma_t**2)))

print(f"Input data shapes:")
print(f"  ts: {ts.shape}")
print(f"  taus: {taus.shape}")
print(f"  gaussian_2d: {gaussian_2d.shape}")
print(f"  Expected: taus along axis 0, ts along axis 1")

In [ ]:
### Test the compute_2d_fft_wavenumber function
nu_ts, nu_taus, data_freq = compute_2d_fft_wavenumber(ts, taus, gaussian_2d)

print(f"\nOutput shapes from compute_2d_fft_wavenumber:")
print(f"  nu_ts: {nu_ts.shape}")
print(f"  nu_taus: {nu_taus.shape}")
print(f"  data_freq: {data_freq.shape}")
print(f"  data_freq dtype: {data_freq.dtype}")

### Analyze the shape reduction
print(f"\nShape analysis:")
print(f"  Input:  ts({N_t}) × taus({N_tau}) → data({N_tau}, {N_t})")
print(f"  Output: nu_ts({len(nu_ts)}) × nu_taus({len(nu_taus)}) → data_freq{data_freq.shape}")
print(f"  Reduction: {N_t}→{len(nu_ts)} (factor: {N_t//len(nu_ts) if len(nu_ts) > 0 else 'N/A'})")
print(f"             {N_tau}→{len(nu_taus)} (factor: {N_tau//len(nu_taus) if len(nu_taus) > 0 else 'N/A'})")

In [ ]:
### Visualize the shape mismatch problem
plt.figure(figsize=(16, 8))

### Plot original 2D Gaussian
plt.subplot(2, 3, 1)
plt.imshow(gaussian_2d, aspect='auto', origin='lower', 
           extent=[ts[0], ts[-1], taus[0], taus[-1]])
plt.colorbar(label='Amplitude')
plt.xlabel(r'$t$ (fs)')
plt.ylabel(r'$\tau$ (fs)')
plt.title(f'Original 2D Gaussian\nShape: {gaussian_2d.shape}')

### Plot FFT magnitude
plt.subplot(2, 3, 2)
plt.imshow(np.abs(data_freq), aspect='auto', origin='lower',
           extent=[nu_ts[0], nu_ts[-1], nu_taus[0], nu_taus[-1]])
plt.colorbar(label='|FFT|')
plt.xlabel(r'$\nu_t$ ($10^4$ cm$^{-1}$)')
plt.ylabel(r'$\nu_{\tau}$ ($10^4$ cm$^{-1}$)')
plt.title(f'FFT Magnitude\nShape: {data_freq.shape}')


plt.tight_layout()
plt.show()

## Test 7: extend_and_plot_results Function

This test creates synthetic 2D spectroscopy data to verify the `extend_and_plot_results` function works correctly and understand its data structure requirements.

In [ ]:
# =============================
# TEST: extend_and_plot_results Function
# =============================
print("\n" + "="*80)
print("TESTING extend_and_plot_results FUNCTION")
print("="*80)

### Create test system parameters
test_system = SystemParameters(
    N_atoms=N_atoms_test,
    ODE_Solver="Paper_eqs",
    RWA_laser=True,
    t_max=100.0,
    dt=0.5,
    Delta_cm=200.0,  # inhomogeneous broadening
    omega_A_cm=16000.0,
    envelope_type='gaussian',
    pulse_fwhm=10.0
)

### Create time arrays
times_test = np.arange(-test_system.fwhms[0], test_system.t_max, test_system.dt)
print(f"Time grid: {len(times_test)} points from {times_test[0]:.1f} to {times_test[-1]:.1f} fs")

### Define multiple T_wait values to test
times_T = np.array([10.0]) #, 30.0, 50.0
print(f"Testing with T_wait values: {times_T}")

### Generate synthetic 2D data for each T_wait
averaged_results = []

for i, T_wait in enumerate(times_T):
    # Get tau and t arrays for this T_wait
    tau_coh_vals, t_det_vals = get_tau_cohs_and_t_dets_for_T_wait(times_test, T_wait)

    if len(tau_coh_vals) == 0 or len(t_det_vals) == 0:
        print(f"  T_wait={T_wait}: No valid time points, skipping")
        averaged_results.append(None)
        continue
        
    print(f"  T_wait={T_wait}: tau_coh={len(tau_coh_vals)} points, t_det={len(t_det_vals)} points")
    
    ### Create synthetic 2D data with realistic structure
    # Data shape should be (N_taus, N_ts) according to the function expectation
    data_shape = (len(tau_coh_vals), len(t_det_vals))
    
    # Create synthetic data with some physical-like features
    tau_mesh, t_mesh = np.meshgrid(tau_coh_vals, t_det_vals, indexing='ij')
    
    # Synthetic 2D spectrum with exponential decay and oscillations
    synthetic_data = (
        np.exp(-tau_mesh / 20.0) *  # coherence decay
        np.exp(-t_mesh / 30.0) *    # population decay  
        np.cos(2 * np.pi * tau_mesh / 6.0) *  # oscillations in tau
        np.sin(2 * np.pi * t_mesh / 10.0)    # oscillations in t
        *(2.0 + 0.1 * np.random.randn(*data_shape))  # small noise
    )
    
    # Add some peak features
    peak_tau_idx = len(tau_coh_vals) // 3
    peak_t_idx = len(t_det_vals) // 3
    synthetic_data[peak_tau_idx-2:peak_tau_idx+3, peak_t_idx-2:peak_t_idx+3] += 0.5
    
    averaged_results.append(synthetic_data.astype(np.float32))
    
    print(f"    Created synthetic data with shape: {synthetic_data.shape}")
    print(f"    Data range: [{synthetic_data.min():.3f}, {synthetic_data.max():.3f}]")

print(f"\nGenerated {len([r for r in averaged_results if r is not None])} valid datasets")

In [ ]:
### Test the extend_and_plot_results function
print("\n" + "-"*60)
print("TESTING extend_and_plot_results function...")
print("-"*60)

### Set up plotting arguments
plot_args_freq = {
    "domain": "freq",
    "type": "real",  # Test with real part
    "save": False,   # Don't save during testing
    "output_dir": None,
    "use_custom_colormap": True,
    "system": test_system,
}

### Test 1: Basic functionality without extension
print("\n1. Testing basic functionality (no extension):")
try:
    extend_and_plot_results(
        averaged_results=averaged_results,
        times_T=times_T,
        times=times_test,
        extend_for=None,  # No extension
        **plot_args_freq
    )
    print("   ✓ Basic functionality test PASSED")
except Exception as e:
    print(f"   ❌ Basic functionality test FAILED: {e}")
    import traceback
    traceback.print_exc()

### small test for SystemParameters

In [ ]:
test_system = SystemParameters(
    N_atoms=N_atoms_test,
    ODE_Solver="Paper_eqs",
    RWA_laser=True,
    omega_laser_cm=16000.0,
    omega_A_cm=17000.0,
    omega_B_cm=16001.0,
)

print("the undiagonalized and diagonalized Hamiltonians are:")
print("H0_undiagonalized:", test_system.H0_undiagonalized)
print("H0_diagonalized:", test_system.H0_diagonalized)

H0 = test_system.H0_diagonalized

### Define time arrays for evolution
t_max = 10
times_test = np.linspace(0, 5 * t_max, 1000)
test_pulse_seq = PulseSequence.from_pulse_specs(test_system, [(0, 0, 0)])
observable_ops = test_system.observable_ops
observable_strs = test_system.observable_strs
H = [H0 + H_int(t, test_pulse_seq, test_system) for t in times_test]

result_me = qt.mesolve(
    test_system.H0,
    test_system.psi_ini,
    times_test,
    e_ops=test_system.observable_ops,
    c_ops=test_system.me_decay_channels,
)

H_int_evo = qt.QobjEvo(lambda t, args=None: matrix_ODE_paper(t, test_pulse_seq, test_system))
result_custom = qt.mesolve(
    H_int_evo,
    test_system.psi_ini,
    times_test,
    e_ops=test_system.observable_ops,
    c_ops=test_system.me_decay_channels,
)

# =============================
# PLOT EVOLUTION
# =============================
plt.figure(figsize=(10, 8))

### Plot result_custom if applicable
if result_custom:
    plt.subplot(3, 1, 1)
    for i, e_op in enumerate(observable_ops):
        plt.plot(
            np.array(result_custom.times) / t_max,
            result_custom.expect[i],
            label=observable_strs[i],
            linestyle=['solid', 'dashed', 'dashdot', 'dotted'][i % 4],  # Alternate line styles
            color=f'C{i}',  # Use distinct colors
        )
    plt.xlabel(r'Time $t / t_{\text{max}}$')
    plt.title(r'Paper_eqs Evolution')

### Plot result_me if applicable
if result_me:
    plt.subplot(3, 1, 2)
    for i, e_op in enumerate(observable_ops):
        plt.plot(
            np.array(result_me.times) / t_max,
            result_me.expect[i],
            label=observable_strs[i],
            linestyle=['solid', 'dashed', 'dashdot', 'dotted'][i % 4],  # Alternate line styles
            color=f'C{i}',  # Use distinct colors
        )
    plt.xlabel(r'Time $t / t_{\text{max}}$')
    plt.title(r'ME Evolution')

plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()